In [ ]:
from __future__ import print_function
from image_helpers import *
from conv_helpers import *
import numpy as np
import scipy.io
import scipy.misc
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

In [ ]:
IMG_CONTENT = '../img/anime_content.jpg'
IMG_STYLE = '../img/noodle_style.png'
IMG_WIDTH = 360
IMG_HEIGHT = 360
IMG_CHANNEL = 3

img_content = scipy.misc.imread(IMG_CONTENT)
img_content = reshape_and_normalize_image(img_content, IMG_WIDTH, IMG_HEIGHT)
# 

img_style = scipy.misc.imread(IMG_STYLE)
img_style = reshape_and_normalize_image(img_style, IMG_WIDTH,IMG_HEIGHT)

img_generated = generate_noise_image(img_content, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNEL)
# img_generated = (np.random.rand(1, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNEL) * 255).astype(int)


In [ ]:
# noise_ratio = 0.6

# noise_image = np.random.uniform(0, 40, (1, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNEL)).astype('float32')
    
# img_generated = (noise_image * noise_ratio + img_content * (1 - noise_ratio)).astype(int)

# img_generated

# img_generated = (np.random.rand(1, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNEL) * 255).astype(int)

# plt.imshow(img_generated[0])

In [ ]:
def show_images(img_content = img_content,
                img_style = img_style,
                img_generated = img_generated):
    fig, ax = plt.subplots(nrows = 1, ncols=3, figsize=(10,10))
    ax[0].imshow(img_content[0])
    ax[0].set_title("CONTENT")
    ax[1].imshow(img_style[0])
    ax[1].set_title("STYLE")
    ax[2].imshow(img_generated[0])
    ax[2].set_title("GENERATED")
    plt.show()
show_images()

In [ ]:
def content_cost(con, gen):
    """
    Calculate the content cost of the generated picture.
    
    Parameters
    ----------
    con: 3d tensor
         The content tensor image.
    
    gen: 3d tensor
         The generated tensor image.
    """
    m, nw, nh, nc = gen.get_shape().as_list()
    con_flatten = tf.reshape(con, [-1])
    gen_flatten = tf.reshape(gen, [-1])
    J_content = tf.reduce_sum((con_flatten - gen_flatten)**2 / (4 * nw * nh * nc))
    return J_content


In [ ]:
tf.reset_default_graph()

with tf.Session() as test:
    tf.set_random_seed(1)
    a_C = tf.random_normal([1, 4, 4, 3], mean=1, stddev=4)
    a_G = tf.random_normal([1, 4, 4, 3], mean=1, stddev=4)
    J_content = content_cost(a_C, a_G)
    print("J_content = " + str(J_content.eval()))


In [ ]:
def gram_matrix(A):
    """
    Calculate the gram matrix of a given input.
    
    Parameters
    ----------
    A: tensor
       The tensor for calculating the gram matrix.
       
    Reference
    ---------
    https://en.wikipedia.org/wiki/Gramian_matrix
    """
    return tf.matmul(A, tf.transpose(A))

In [ ]:
tf.reset_default_graph()

with tf.Session() as test:
    tf.set_random_seed(1)
    A = tf.random_normal([3, 2*1], mean=1, stddev=4)
    GA = gram_matrix(A)
    
    print("GA = " + str(GA.eval()))

In [ ]:
def style_cost(sty, gen):
    """
    Calculate the style cost of a single layer the generated picture.
    
    Parameters
    ----------
    sty: 3d tensor
         The style tensor image.
    
    gen: 3d tensor
         The generated tensor image.
    """
    m, nw, nh, nc = gen.get_shape().as_list()
    sty_res = tf.transpose(tf.reshape(sty, (nw * nh, nc)))
    gen_res = tf.transpose(tf.reshape(gen, (nw * nh, nc)))
    gm_sty = gram_matrix(sty_res)
    gm_gen = gram_matrix(gen_res)
    sty_flatten = tf.reshape(gm_sty, [-1])
    gen_flatten = tf.reshape(gm_gen, [-1])
    J_style = tf.reduce_sum((sty_flatten - gen_flatten)**2 / (4 * nc**2 * (nh * nw)**2))
    return J_style

In [ ]:
tf.reset_default_graph()

with tf.Session() as test:
    tf.set_random_seed(1)
    a_S = tf.random_normal([1, 4, 4, 3], mean=1, stddev=4)
    a_G = tf.random_normal([1, 4, 4, 3], mean=1, stddev=4)
    J_style_layer = style_cost(a_S, a_G)
    
    print("J_style_layer = " + str(J_style_layer.eval()))

In [ ]:
STYLE_LAYERS = [
    ('conv1', 0.2),
    ('conv2', 0.2),
    ('conv3', 0.2),
    ('conv4', 0.2),
    ('conv5', 0.2)]

In [ ]:
def full_style_cost(model, sess = tf.get_default_session(), STYLE_LAYERS=STYLE_LAYERS):
    J_style = 0
    
    for layer_name, cost in STYLE_LAYERS:
        out = model[layer_name]
        a_S = sess.run(out)
        a_G = out
        
        J_style_layer = style_cost(a_S, a_G)
        J_style += cost * J_style_layer
        
    return J_style

In [ ]:
def total_cost(J_content, J_style, alpha = 10, beta = 40):
    """
    Calculate the total cost of neural style transfer.
    
    Parameters
    ----------
    J_content: float
        The content cost.
    
    J_style: float
        The style cost.
    
    alpha: double
        Hyperparameter. J_content weight.
        
    beta: double
        Hyperparameter. J_style weight.
    """
    J = alpha * J_content + beta * J_style
    return J

In [ ]:
tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(3)
    J_content = np.random.randn()    
    J_style = np.random.randn()
    J = total_cost(J_content, J_style)
    print("J = " + str(J))

In [ ]:
# Hyperparameters
learning_rate = 0.001
num_classes = 1000
dropout_rate = 0.5
batch_size = 1

In [ ]:
# tf.reset_default_graph()
# x = tf.placeholder(tf.float32, [batch_size, 360, 360, 3])
# x = tf.Variable(np.zeros((1, IMG_HEIGHT, IMG_WIDTH, 3)), dtype='float32')
# keep_prob = tf.placeholder(tf.float32)

In [ ]:
tf.reset_default_graph()
model = export_graph(IMG_WIDTH, IMG_HEIGHT, IMG_CHANNEL)
sess = tf.InteractiveSession()
sess.run(model['input'].assign(img_content))
out = model['conv4']
a_C = sess.run(out)
a_G = out
J_content = content_cost(a_C, a_G)
print(type(a_G), type(a_C), type(J_content))

In [ ]:
sess.run(model['input'].assign(img_style))
J_style = full_style_cost(model, sess=sess)
J_style

In [ ]:
J = total_cost(J_content, J_style)
J

In [ ]:
optimizer = tf.train.AdamOptimizer(2.0)
train_step = optimizer.minimize(J)

In [ ]:
def train(input_image, model = model, sess = tf.get_default_session(), num_iter=200):
    sess.run(tf.global_variables_initializer())
    sess.run(model['input'].assign(input_image))
    
    for i in range(num_iter):
        _ = sess.run(train_step)
        generated_image = sess.run(model['input'])
        
        if i % 20 == 0:
            jt, jc, js = sess.run([J, J_content, J_style])
            print("Iteration: ", i)
            print("Total cost: ", str(jt))
            print("Content cost: ", str(jc))
            print("Style cost: ", str(js))
            
            save_image("output/" + str(i) + ".png", generated_image)
            
    save_image('generated_image.jpg', generated_image)
    
    return generated_image

In [ ]:
result = train(img_generated, model, sess, num_iter=800)

In [ ]:
show_images(img_generated = result.astype(int))